In [1]:
import glob
import json
import sys

import numpy as np

sys.path.append('contrastive')

from contrastive.eval.common import non_iid_bound

In [2]:
seeds = [7, 11, 13]
num_seeds = len(seeds)

## Table 3

In [3]:
model = 'non-iid'
score_types = ['top', 'avg']

algorithms = [
    'stochastic',
    'deterministic',
    'pac-bayes',
    'arora',
    'catoni-stochastic',
    'catoni-deterministic',
    'catoni-pac-bayes'
]


table_data = {}
for algorithm in algorithms:
    results = {'top1': [], 'top5': [], 'mu5-top1': [], 'mu5-top5': [], 'avg2': [], 'mu5-avg2': []}    
    for score_type in score_types:
        for seed in seeds:
            fname = 'results/{}/{}-{}-{}.json'.format(model, algorithm, score_type, seed)
            data = json.load(open(fname))
            for k, v in data.items():
                results[k].append(v)
    table_data[algorithm] = results


avg2_row = '\\avgtwo  & '
top1_row = '\\topone  & '
top5_row = '\\topfive & '

score_format = '${:.1f}$'.format
for k, data in table_data.items():

    top1_row += score_format(np.array(data['top1']).mean())
    top1_row += ' & '
    top1_row += score_format(np.array(data['mu5-top1']).mean())    

    top5_row += score_format(np.array(data['top5']).mean())
    top5_row += ' & '
    top5_row += score_format(np.array(data['mu5-top5']).mean())

    avg2_row += score_format(np.array(data['avg2']).mean())
    avg2_row += ' & '
    avg2_row += score_format(np.array(data['mu5-avg2']).mean())

    if k == 'pac-bayes':
        separator = ' &&& '
    else:
        separator = ' && '
    top1_row += separator
    top5_row += separator
    avg2_row += separator
    
print('\\\\ \n'.join([avg2_row[:-3], top1_row[:-3], top5_row[:-3]]) + '\\\\')


\avgtwo  & $69.2$ & $64.0$ && $69.7$ & $64.3$ && $69.4$ & $64.2$ &&& $75.2$ & $67.3$ && $74.6$ & $66.4$ && $74.8$ & $66.5$ && $72.3$ & $64.0$ \\ 
\topone  & $6.8$ & $4.9$ && $6.9$ & $4.9$ && $6.9$ & $5.0$ &&& $20.5$ & $9.6$ && $19.6$ & $9.1$ && $21.0$ & $9.3$ && $14.4$ & $7.5$ \\ 
\topfive & $22.0$ & $17.1$ && $22.1$ & $17.3$ && $22.5$ & $17.3$ &&& $41.7$ & $23.4$ && $40.8$ & $22.8$ && $41.7$ & $22.9$ && $34.9$ & $20.1$ \\


In [4]:
def get_best_model_name(model_name_dir, criterion_key_in_json='lowest_val_loss'):

    if model_name_dir[-1] == '/':
        model_name_dir = model_name_dir[:-1]
    fnames = glob.glob('{}/*.json'.format(model_name_dir))

    lowest_val_loss = np.finfo(np.float(0.)).max

    best_model_fname = ''
    for fname in fnames:
        result = json.load(open(fname))
        if criterion_key_in_json not in result:
            print('{} field is not found in {}'.format(criterion_key_in_json, fname))
            continue

        if result[criterion_key_in_json] < lowest_val_loss:
            lowest_val_loss = result[criterion_key_in_json]
            best_model_fname = fname

    return best_model_fname.replace('json', 'pt')


In [5]:
float_format = ' ${:.3f}$ &'.format
line_break = '\\\\'

## Table 4: bounds on NAUSLAN

In [6]:
val_algorithms = [
    'stochastic',
    'deterministic',
]

results = []

chi_square_format = ' ${:.3f}$ &'.format

empty_cell = ' -- &'

model = 'non-iid'

emp_risk_row = '$\\Runhat(\\Qcal)$ &'
test_risk_row = '$\\Run(\\Qcal)$ &'
emp_risk_f_row = '$\\Runhat(\\fbf^*)$ &'
test_risk_f_row = '$\\Run(\\fbf^*)$ &'
bound_row = 'Bound &'
chi_row = '$\\chi^2$ &'

# calculate PAC-Bayes bound and risks for best stochastic/deterministic models with respect to validation risk.
for algorithm in val_algorithms:
    emp_risk = 0. 
    test_risk = 0.
    emp_risk_f = 0. 
    test_risk_f = 0.
    bound = 0.
    chi_square = 0.

    for seed in seeds:
        fname = get_best_model_name(model_name_dir='weights/{}/{}/seed-{}'.format(model, algorithm, seed))
        data = json.load(open('./bounds/{}/pac-bayes-{}-{}.json'.format(model, seed, algorithm)))
        terms = data[fname]

        terms_f = json.load(open('./bounds/{}/pac-bayes-{}-{}-det.json'.format(model, seed, algorithm)))[fname]
        emp_risk_f += terms_f['train-zero-one-loss']
        test_risk_f += terms_f['test-zero-one-loss']
        del terms_f

        pb_bound = non_iid_bound(terms)

        print('\r', model, algorithm, seed, pb_bound, end='')

        emp_risk += terms['train-zero-one-loss']
        test_risk += terms['test-zero-one-loss']
        bound += pb_bound
        chi_square +=  terms['chi_square']

    emp_risk_row += float_format(emp_risk / num_seeds)
    test_risk_row +=  float_format(test_risk / num_seeds)
    emp_risk_f_row += float_format(emp_risk_f / num_seeds)
    test_risk_f_row +=  float_format(test_risk_f / num_seeds)        
    bound_row += float_format(bound / num_seeds)
    chi_row += chi_square_format(chi_square / num_seeds)

# calculate PAC-Bayes bound and risks for best models with respect to the PAC-Bayes bound.
emp_risk = 0.
test_risk = 0.
emp_risk_f = 0.
test_risk_f = 0.    
bound = 0.
chi_square = 0.

for seed in seeds:
    data = json.load(open('./bounds/{}/pac-bayes-{}.json'.format(model, seed)))
    data_f = json.load(open('./bounds/{}/pac-bayes-{}-det.json'.format(model, seed)))

    best_terms = {}
    best_terms_f = {}
    best_bound = np.finfo(np.float(0.)).max
    for k, terms in data.items():
        pb_bound = non_iid_bound(terms)
        if pb_bound < best_bound:
            best_terms = terms
            best_bound = pb_bound
            best_terms_f = data_f[k]

    print('\r', model, 'PAC-Bayes', seed, best_bound, end='')

    emp_risk += best_terms['train-zero-one-loss']
    test_risk += best_terms['test-zero-one-loss']
    emp_risk_f += best_terms_f['train-zero-one-loss']
    test_risk_f += best_terms_f['test-zero-one-loss']
    bound += best_bound
    chi_square +=  best_terms['chi_square']

print('\r', end='')

separator = '\\\\ '
emp_risk_row += float_format(emp_risk / num_seeds) + separator
test_risk_row += float_format(test_risk / num_seeds ) + separator
emp_risk_f_row += float_format(emp_risk_f / num_seeds) + separator
test_risk_f_row += float_format(test_risk_f / num_seeds ) + separator
bound_row += float_format(bound / num_seeds) + separator
chi_row += chi_square_format( chi_square / num_seeds) + separator
results.append('\n'.join([emp_risk_f_row, test_risk_f_row, emp_risk_row, test_risk_row, bound_row, chi_row]))


In [7]:
print(''.join(results).replace(',', '\\,').replace('&\\\\', '\\\\'))

$\Runhat(\fbf^*)$ & $0.056$ & $0.073$ & $0.033$ \\ 
$\Run(\fbf^*)$ & $0.089$ & $0.103$ & $0.074$ \\ 
$\Runhat(\Qcal)$ & $0.058$ & $0.065$ & $0.058$ \\ 
$\Run(\Qcal)$ & $0.106$ & $0.108$ & $0.097$ \\ 
Bound & $4.460$ & $140.949$ & $2.227$ \\ 
$\chi^2$ & $0.012$ & $980.354$ & $0.052$ \\ 
